# Job recommendation system



recommendation system for given dataset



*   content based recommendation system is used to recommend jobs in this dataset.
*   A Content-Based Recommender works by the data that we take from the user, either explicitly or implicitly . By the data we create a user profile, which is then used to suggest to the user, as the user provides more input or take more actions on the recommendation, the engine becomes more accurate.



*   Data is tokenized and sum of the cosine distance over all tokens is calculated and used to find the cosine similarity.




In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/nlp/Combined_Jobs_Final.csv")
df = df[~df['Job.Description'].isna()]
df.rename(columns={'Job.Description':'sentence'}, inplace=True)
print(len(df))
df = df.iloc[:20000]
df.head()

84034


,Job.ID,Provider,Status,Slug,Title,Position,Company,City,State.Name,State.Code,...,Industry,sentence,Requirements,Salary,Listing.Start,Listing.End,Employment.Type,Education.Required,Created.At,Updated.At
0,111,1,open,palo-alto-ca-tacolicious-server,Server @ Tacolicious,Server,Tacolicious,Palo Alto,California,CA,...,Food and Beverages,Tacolicious' first Palo Alto store just opened...,NaN,8.00,NaN,NaN,Part-Time,NaN,2013-03-12 02:08:28 UTC,2014-08-16 15:35:36 UTC
1,113,1,open,san-francisco-ca-claude-lane-kitchen-staff-chef,Kitchen Staff/Chef @ Claude Lane,Kitchen Staff/Chef,Claude Lane,San Francisco,California,CA,...,Food and Beverages,\r\n\r\nNew French Brasserie in S.F. Financia...,NaN,0.00,NaN,NaN,Part-Time,NaN,2013-04-12 08:36:36 UTC,2014-08-16 15:35:36 UTC
2,117,1,open,san-francisco-ca-machka-restaurants-corp-barte...,Bartender @ Machka Restaurants Corp.,Bartender,Machka Restaurants Corp.,San Francisco,California,CA,...,Food and Beverages,We are a popular Mediterranean wine bar and re...,NaN,11.00,NaN,NaN,Part-Time,NaN,2013-07-16 09:34:10 UTC,2014-08-16 15:35:37 UTC
3,121,1,open,brisbane-ca-teriyaki-house-server,Server @ Teriyaki House,Server,Teriyaki House,Brisbane,California,CA,...,Food and Beverages,● Serve food/drinks to customers in a profess...,NaN,10.55,NaN,NaN,Part-Time,NaN,2013-09-04 15:40:30 UTC,2014-08-16 15:35:38 UTC
4,127,1,open,los-angeles-ca-rosa-mexicano-sunset-kitchen-st...,Kitchen Staff/Chef @ Rosa Mexicano - Sunset,Kitchen Staff/Chef,Rosa Mexicano - Sunset,Los Angeles,California,CA,...,Food and Beverages,"Located at the heart of Hollywood, we are one ...",NaN,10.55,NaN,NaN,Part-Time,NaN,2013-07-17 15:26:18 UTC,2014-08-16 15:35:40 UTC


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20000 entries, 0 to 20006
Data columns (total 23 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Job.ID              20000 non-null  int64  
 1   Provider            20000 non-null  int64  
 2   Status              20000 non-null  object 
 3   Slug                20000 non-null  object 
 4   Title               20000 non-null  object 
 5   Position            20000 non-null  object 
 6   Company             19272 non-null  object 
 7   City                19987 non-null  object 
 8   State.Name          19973 non-null  object 
 9   State.Code          19973 non-null  object 
 10  Address             15 non-null     object 
 11  Latitude            20000 non-null  float64
 12  Longitude           20000 non-null  float64
 13  Industry            106 non-null    object 
 14  sentence            20000 non-null  object 
 15  Requirements        0 non-null      float64
 16  Sala

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re

STOPWORDS = set(stopwords.words('english'))
MIN_WORDS = 4
MAX_WORDS = 200

PATTERN_S = re.compile("\'s")  # matches `'s` from text
PATTERN_RN = re.compile("\\r\\n") #matches `\r` and `\n`
PATTERN_PUNC = re.compile(r"[^\w\s]") # matches all non 0-9 A-z whitespace

def clean_text(text):
    """
    Series of cleaning. String to lower case, remove non words characters and numbers.
        text (str): input text
    return (str): modified initial text
    """
    text = text.lower()  # lowercase text
    text = re.sub(PATTERN_S, ' ', text)
    text = re.sub(PATTERN_RN, ' ', text)
    text = re.sub(PATTERN_PUNC, ' ', text)
    return text

def tokenizer(sentence, min_words=MIN_WORDS, max_words=MAX_WORDS, stopwords=STOPWORDS, lemmatize=True):
    """
    Lemmatize, tokenize, crop and remove stop words.
    """
    if lemmatize:
        stemmer = WordNetLemmatizer()
        tokens = [stemmer.lemmatize(w) for w in word_tokenize(sentence)]
    else:
        tokens = [w for w in word_tokenize(sentence)]
    token = [w for w in tokens if (len(w) > min_words and len(w) < max_words
                                                        and w not in stopwords)]
    return tokens


def clean_sentences(df):
    """
    Remove irrelavant characters (in new column clean_sentence).
    Lemmatize, tokenize words into list of words (in new column tok_lem_sentence).
    """
    print('Cleaning sentences...')
    df['clean_sentence'] = df['sentence'].apply(clean_text)
    df['tok_lem_sentence'] = df['clean_sentence'].apply(
        lambda x: tokenizer(x, min_words=MIN_WORDS, max_words=MAX_WORDS, stopwords=STOPWORDS, lemmatize=True))
    return df

df = clean_sentences(df)

Cleaning sentences...


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20000 entries, 0 to 20006
Data columns (total 25 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Job.ID              20000 non-null  int64  
 1   Provider            20000 non-null  int64  
 2   Status              20000 non-null  object 
 3   Slug                20000 non-null  object 
 4   Title               20000 non-null  object 
 5   Position            20000 non-null  object 
 6   Company             19272 non-null  object 
 7   City                19987 non-null  object 
 8   State.Name          19973 non-null  object 
 9   State.Code          19973 non-null  object 
 10  Address             15 non-null     object 
 11  Latitude            20000 non-null  float64
 12  Longitude           20000 non-null  float64
 13  Industry            106 non-null    object 
 14  sentence            20000 non-null  object 
 15  Requirements        0 non-null      float64
 16  Sala

In [ ]:
print(len(df))
df[['sentence', 'clean_sentence', 'tok_lem_sentence']]

20000


,sentence,clean_sentence,tok_lem_sentence
0,Tacolicious' first Palo Alto store just opened...,tacolicious first palo alto store just opened...,"[tacolicious, first, palo, alto, store, just, ..."
1,\r\n\r\nNew French Brasserie in S.F. Financia...,new french brasserie in s f financial dist...,"[new, french, brasserie, in, s, f, financial, ..."
2,We are a popular Mediterranean wine bar and re...,we are a popular mediterranean wine bar and re...,"[we, are, a, popular, mediterranean, wine, bar..."
3,● Serve food/drinks to customers in a profess...,serve food drinks to customers in a profess...,"[serve, food, drink, to, customer, in, a, prof..."
4,"Located at the heart of Hollywood, we are one ...",located at the heart of hollywood we are one ...,"[located, at, the, heart, of, hollywood, we, a..."
...,...,...,...
20002,\r\nYou belong at Dayton Freight! Join our tea...,you belong at dayton freight join our team o...,"[you, belong, at, dayton, freight, join, our, ..."
20003,"Lorien Bel Air&nbsp;is a clean, modern Assiste...",lorien bel air nbsp is a clean modern assiste...,"[lorien, bel, air, nbsp, is, a, clean, modern,..."
20004,\r\n\r\nESSENTIAL DUTIES AND RESPONSIBILITIES:...,essential duties and responsibilities nbsp...,"[essential, duty, and, responsibility, nbsp, b..."
20005,Part TimeEmertius at Canterbury Ridge- 1706 E....,part timeemertius at canterbury ridge 1706 e ...,"[part, timeemertius, at, canterbury, ridge, 17..."


## Query sentence

In [ ]:
query_sentence = "Job in california for a person with Masters Degree"

pd.options.display.max_colwidth = 500

## Util function

In [ ]:
def extract_best_indices(m, topk, mask=None):
    """
    Use sum of the cosine distance over all tokens.
    m (np.array): cos matrix of shape (nb_in_tokens, nb_dict_tokens)
    topk (int): number of indices to return (from high to lowest in order)
    """
    # return the sum on all tokens of cosinus for each sentence
    if len(m.shape) > 1:
        cos_sim = np.mean(m, axis=0)
    else:
        cos_sim = m
    index = np.argsort(cos_sim)[::-1] # from highest idx to smallest score
    if mask is not None:
        assert mask.shape == m.shape
        mask = mask[index]
    else:
        mask = np.ones(len(cos_sim))
    mask = np.logical_or(cos_sim[index] != 0, mask) #eliminate 0 cosine distance
    best_index = index[mask][:topk]
    return best_index

TF-IDF Training


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Adapt stop words
token_stop = tokenizer(' '.join(STOPWORDS), lemmatize=False)

# Fit TFIDF
vectorizer = TfidfVectorizer(stop_words=token_stop, tokenizer=tokenizer)
tfidf_mat = vectorizer.fit_transform(df['sentence'].values) # -> (num_sentences, num_vocabulary)
tfidf_mat.shape

/usr/local/lib/python3.9/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/feature_extraction/text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['doe', 'ha', 'wa'] not in stop_words.
  warnings.warn(


(20000, 54034)

In [ ]:
#Prediction
def get_recommendations_tfidf(sentence, tfidf_mat):

    """
    Return the database sentences in order of highest cosine similarity relatively to each
    token of the target sentence.
    """
    # Embed the query sentence
    tokens = [str(tok) for tok in tokenizer(sentence)]
    vec = vectorizer.transform(tokens)
    # Create list with similarity between query and dataset
    mat = cosine_similarity(vec, tfidf_mat)
    # Best cosine distance for each token independantly
    print(mat.shape)
    best_index = extract_best_indices(mat, topk=3)
    return best_index


best_index = get_recommendations_tfidf(query_sentence, tfidf_mat)

display(df[['State.Name', 'Education.Required', 'sentence']].iloc[best_index])

(9, 20000)


,State.Name,Education.Required,sentence
1874,Georgia,Not Specified,"Ref ID:00850-9715035Classification:Account Executive/Staffing ManagerCompensation:DOESavannah logistics company is seeking a Purchasing Manager for a long-term opportunity. Job duties will include: Basic demand planning duties, setting up reorder points, basic cost duties, loading master data, publishing master data, loading quotes from the supplier, must be able to work overtime during month end and quarter end. The ideal candidate would have 5 years of supply chain experience or a degree i..."
16373,Illinois,Not Specified,"Ref ID:01300-9731321Classification:Credit/CollectionsCompensation:DOEOur client, located in the Northwest suburbs, is looking to add a client services candidate to their staff for a temp to perm role. This person must have 2+ years of financial services experience and strong excel skills. This person will be responsible for communicating with clients and brokers on a daily basis. This person will be responsible for reviewing and processing cerfitificates of insurance. This person must be ver..."
9263,New York,High School Diploma,"Independent auto repair shop in beautiful Buffalo NY is looking for a Experienced Master automotive Technician! Do you have the talent and the ability? We need an Automotive master technician that can diagnose the difficult problems and help run the shop with a good attitude, with an opportunity to excel with in the company."


In [ ]:
#Load pretrained SPACY

In [ ]:
!python -m spacy download en_core_web_lg


2023-04-19 19:29:15.368172: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-19 19:29:16.474953: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 2.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [ ]:
import spacy



#Load pre-trained model
nlp = spacy.load("en_core_web_lg")
# Apply the model to the sentences
df['spacy_sentence'] = df['sentence'].apply(lambda x: nlp(x))
# Retrieve the embedded vectors as a matrix
embed_mat = df['spacy_sentence'].values
embed_mat.shape

(20000,)

In [ ]:
def predict_spacy(model, query_sentence, embed_mat, topk=5):
    """
    Predict the topk sentences after applying spacy model.
    """
    query_embed = model(query_sentence)
    mat = np.array([query_embed.similarity(line) for line in embed_mat])
    # keep if vector has a norm
    mat_mask = np.array(
        [True if line.vector_norm else False for line in embed_mat])
    best_index = extract_best_indices(mat, topk=topk, mask=mat_mask)
    return best_index

# Predict
predict_spacy(nlp, query_sentence, embed_mat)
display(df[['State.Name', 'Education.Required', 'sentence']].iloc[best_index])

<ipython-input-17-551accf314df>:6: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  mat = np.array([query_embed.similarity(line) for line in embed_mat])


,State.Name,Education.Required,sentence
1874,Georgia,Not Specified,"Ref ID:00850-9715035Classification:Account Executive/Staffing ManagerCompensation:DOESavannah logistics company is seeking a Purchasing Manager for a long-term opportunity. Job duties will include: Basic demand planning duties, setting up reorder points, basic cost duties, loading master data, publishing master data, loading quotes from the supplier, must be able to work overtime during month end and quarter end. The ideal candidate would have 5 years of supply chain experience or a degree i..."
16373,Illinois,Not Specified,"Ref ID:01300-9731321Classification:Credit/CollectionsCompensation:DOEOur client, located in the Northwest suburbs, is looking to add a client services candidate to their staff for a temp to perm role. This person must have 2+ years of financial services experience and strong excel skills. This person will be responsible for communicating with clients and brokers on a daily basis. This person will be responsible for reviewing and processing cerfitificates of insurance. This person must be ver..."
9263,New York,High School Diploma,"Independent auto repair shop in beautiful Buffalo NY is looking for a Experienced Master automotive Technician! Do you have the talent and the ability? We need an Automotive master technician that can diagnose the difficult problems and help run the shop with a good attitude, with an opportunity to excel with in the company."


# Word2Vec

In [ ]:
from gensim.models.word2vec import Word2Vec

# Create model
word2vec_model = Word2Vec(min_count=0, workers = 8, vector_size=300)
# Prepare vocab
word2vec_model.build_vocab(df.tok_lem_sentence.values)
# Train
word2vec_model.train(df.tok_lem_sentence.values, total_examples=word2vec_model.corpus_count, epochs=30)

(104753353, 132865980)

## Predict

In [ ]:
def is_word_in_model(word, model):
    """
    Check on individual words ``word`` that it exists in ``model``.
    """
    assert type(model).__name__ == 'KeyedVectors'
    is_in_vocab = word in model.key_to_index.keys()
    return is_in_vocab

def predict_w2v(query_sentence, dataset, model, topk=3):
    query_sentence = query_sentence.split()
    in_vocab_list, best_index = [], [0]*topk
    for w in query_sentence:
        # remove unseen words from query sentence
        if is_word_in_model(w, model.wv):
            in_vocab_list.append(w)
    # Retrieve the similarity between two words as a distance
    if len(in_vocab_list) > 0:
        sim_mat = np.zeros(len(dataset))  # TO DO
        for i, data_sentence in enumerate(dataset):
            if data_sentence:
                sim_sentence = model.wv.n_similarity(
                        in_vocab_list, data_sentence)
            else:
                sim_sentence = 0
            sim_mat[i] = np.array(sim_sentence)
        # Take the five highest norm
        best_index = np.argsort(sim_mat)[::-1][:topk]
    return best_index

# Predict
best_index = predict_w2v(query_sentence, df['tok_lem_sentence'].values, word2vec_model)
display(df[['State.Name', 'Education.Required', 'sentence']].iloc[best_index])

,State.Name,Education.Required,sentence
5223,Georgia,Bachelor's Degree,"\r\nQUALIFIED CANDIDATES MUST SUBMIT A RESUME WITH THE CAREERBUILDER APPLICATION!&nbsp; THIS POSITION IS 20 HOURS PER WEEK 5 HRS PER DAY WITH LIMITED FLEXIBILITY.\r\n\r\n\r\nFounded in 1995, Foundations Recovery Network (FRN) quickly established itself as a premier organization for providing integrated treatment for persons with co-occurring mental health and substance use disorders.&nbsp; FRN offers a continuum of care including outpatient services and residential programs in Nashville &amp..."
6116,Pennsylvania,Not Specified,"The Renfrew Center of Philadelphia, PA has an immediate opening for a Research Intern. The Renfrew Center has been a pioneer in the treatment of eating disorders since 1985. As the nation’s first residential eating disorder facility, now with 16 locations throughout the country, Renfrew has helped more than 65,000 women with eating disorders and other behavioral health issues. Renfrew provides women suffering from anorexia nervosa, bulimia nervosa, binge eating disorder, eating disorder no..."
7926,Indiana,High School Diploma,"Easter Seals Crossroads is an Equal Opportunity/Affirmative Action employer. All qualified applicants will receive consideration for employment without regard to race, color, religion, sex, national origin, disability, sexual orientation, gender identity, protected Veteran status, or any other legally protected status.\r\n\r\nAre you energetic, outgoing, and have a passion for mentoring others? Easter Seals Crossroads is hiring people with a passion for mentoring others! We have part-time an..."


# Transformers

## Load pretrained

In [ ]:
!pip install sentence_transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 45.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 56.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 53.3 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125942 sha256=3de132029cb4f1733d14f5935ac12aee1b3f8dc751fa63201977ad5f5beb09da
  Stored in directory: /root/.cache/pip/wheels/71/67/06/162a3760c40d74dd40bc855d527008d26341c2b0ecf3e8e11f
Successfully built sentence_transformers


In [ ]:
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer('paraphrase-MiniLM-L6-v2')
corpus_embeddings = model.encode(df.sentence.values, convert_to_tensor=True)
query_embedding = model.encode(query_sentence, convert_to_tensor=True)


## Predict

In [ ]:
import torch

# We use cosine-similarity and torch.topk to find the highest 3 scores
cos_scores = util.pytorch_cos_sim(query_embedding, corpus_embeddings)[0]
top_results = torch.topk(cos_scores, k=3)

print("\n\n======================\n\n")
print("Query:", query_sentence)
print("\nTop 5 most similar sentences in corpus:")

for score, idx in zip(top_results[0], top_results[1]):
    score = score.cpu().data.numpy()
    idx = idx.cpu().data.numpy()
    display(df[['State.Name', 'Education.Required', 'sentence']].iloc[idx])





Query: Job in california for a person with Masters Degree

Top 5 most similar sentences in corpus:


State.Name                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     California
Education.Required                                                                                                                                                                                                                                                                                                                                                                                                                                                                            

State.Name                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      California
Education.Required                                                                                                                                                                                                                                                                                                                                                                                                                                                                           Not Specified
sentence          

State.Name                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     California
Education.Required                                                                                                                                                                                                                                                                                                                                                                                                                                                                            

**Observations**:


*   The query sentence is Job in california for a person with Masters Degree
*   Reccommendation gave outputs which have most similarity first output have california which is present in the query.
*  Using sentence_transformers gave better results .

